<a href="https://colab.research.google.com/github/Superzzzz0933/generative-ai-for-beginners/blob/main/Quora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 使用 wget 命令从网络上直接下载文件到 Colab 的临时存储中
!wget https://sbert.net/datasets/quora_duplicate_questions.tsv.zip

# 解压缩刚刚下载的 zip 文件
!unzip quora_duplicate_questions.tsv.zip

# 查看一下当前目录下的文件，确认我们得到了 .tsv 文件
!ls -lh

--2025-07-07 09:06:14--  https://sbert.net/datasets/quora_duplicate_questions.tsv.zip
Resolving sbert.net (sbert.net)... 104.21.80.1, 104.21.48.1, 104.21.64.1, ...
Connecting to sbert.net (sbert.net)|104.21.80.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://public.ukp.informatik.tu-darmstadt.de/reimers/sentence-transformers/datasets/quora_duplicate_questions.tsv.zip [following]
--2025-07-07 09:06:15--  https://public.ukp.informatik.tu-darmstadt.de/reimers/sentence-transformers/datasets/quora_duplicate_questions.tsv.zip
Resolving public.ukp.informatik.tu-darmstadt.de (public.ukp.informatik.tu-darmstadt.de)... 130.83.167.186
Connecting to public.ukp.informatik.tu-darmstadt.de (public.ukp.informatik.tu-darmstadt.de)|130.83.167.186|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2025-07-07 09:06:16 ERROR 404: Not Found.

unzip:  cannot find or open quora_duplicate_questions.tsv.zip, quora_duplicate_questions.tsv.

In [2]:
# 导入 Colab 的文件处理工具
from google.colab import files
import os

print('请上传您刚刚下载的 kaggle.json 文件：')

# 1. 上传 kaggle.json 文件
# 运行这行后，会出现一个文件选择对话框，请选择你下载的 kaggle.json
uploaded = files.upload()

# 2. 配置 Kaggle API 的路径
# 我们需要把这个 json 文件放到它应该在的位置，并设置好权限
if 'kaggle.json' in uploaded:
    print("kaggle.json 上传成功！正在配置环境...")
    # 创建一个 .kaggle 目录
    !mkdir -p ~/.kaggle
    # 将 json 文件移动到该目录下
    !mv kaggle.json ~/.kaggle/
    # 设置文件权限为 600，表示只有你自己可以读写，这是安全必需的
    !chmod 600 ~/.kaggle/kaggle.json
else:
    print("未找到 kaggle.json，请重新运行此单元格并上传文件。")


# 3. 使用 Kaggle API 下载数据集
# 这是从 kaggle 下载 quora-question-pairs 数据集的官方命令
!kaggle datasets download -d quora/question-pairs-dataset

# 4. 解压缩下载的文件
# Kaggle 下载的文件名通常是'xxx.zip'
!unzip -o question-pairs-dataset.zip

# 5. 验证文件是否已准备好
# 我们应该能看到一个名为 questions.csv 的文件
!ls -lh

请上传您刚刚下载的 kaggle.json 文件：


Saving kaggle.json to kaggle.json
kaggle.json 上传成功！正在配置环境...
Dataset URL: https://www.kaggle.com/datasets/quora/question-pairs-dataset
License(s): other
  0% 0.00/20.8M [00:00<?, ?B/s]
100% 20.8M/20.8M [00:00<00:00, 1.25GB/s]
Archive:  question-pairs-dataset.zip
  inflating: questions.csv           
total 79M
-rw-r--r-- 1 root root  21M Sep 20  2019 question-pairs-dataset.zip
-rw-r--r-- 1 root root  58M Sep 20  2019 questions.csv
drwxr-xr-x 1 root root 4.0K Jul  1 21:04 sample_data


In [3]:
import pandas as pd

# 加载新的 CSV 文件
# 这次是标准的逗号分隔文件，所以我们不需要 'sep' 参数
df = pd.read_csv('questions.csv')

# 再次进行预览和检查
print("--- 数据预览 (前5行) ---")
print(df.head())

print("\n--- 数据基本信息 ---")
df.info()

--- 数据预览 (前5行) ---
   id  qid1  qid2                                          question1  \
0   0     1     2  What is the step by step guide to invest in sh...   
1   1     3     4  What is the story of Kohinoor (Koh-i-Noor) Dia...   
2   2     5     6  How can I increase the speed of my internet co...   
3   3     7     8  Why am I mentally very lonely? How can I solve...   
4   4     9    10  Which one dissolve in water quikly sugar, salt...   

                                           question2  is_duplicate  
0  What is the step by step guide to invest in sh...             0  
1  What would happen if the Indian government sto...             0  
2  How can Internet speed be increased by hacking...             0  
3  Find the remainder when [math]23^{24}[/math] i...             0  
4            Which fish would survive in salt water?             0  

--- 数据基本信息 ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404351 entries, 0 to 404350
Data columns (total 6 columns):
 #   Col

In [5]:
import pandas as pd

# 为了确保我们使用的是上一步加载的数据，我们重新运行加载数据的代码
# 如果你的Colab环境没有中断，也可以不运行这行，但这是一个好习惯
df = pd.read_csv('questions.csv') # Removed redundant file loading

# --- 1. 清洗数据：处理缺失值 ---
print(f"清洗前的数据总行数: {len(df)}")

# 检查 question1 或 question2 列是否有空值 (NaN)
# .dropna() 会移除任何包含空值的行
# subset=['question1', 'question2'] 表示我们只关心这两列的空值
# inplace=True 表示直接在原始 DataFrame 上修改，不创建副本，节省内存
df.dropna(subset=['question1', 'question2'], inplace=True)

print(f"清洗后的数据总行数: {len(df)}")


# --- 2. 构建语料库 (Corpus) ---
# 我们的目标是建立一个可以被搜索的“问题数据库”。
# 这个数据库应该包含数据集中所有出现过的问题，且不重复。

# 首先，将 'question1' 和 'question2' 两列的所有问题合并到一个 pandas Series 对象中
all_questions = pd.concat([df['question1'], df['question2']])

# 然后，使用 .unique() 方法找出所有不重复的问题
unique_questions = all_questions.unique()

# 最后，将结果转换成一个 Python 列表，这就是我们的语料库
corpus = unique_questions.tolist()

print(f"\n成功构建语料库！包含 {len(corpus)} 个不重复的问题。")


# --- 3. 为快速实践，截取部分语料库 ---
# 完整语料库有超过50万个问题，全部编码需要较长时间（约20-30分钟）。
# 为了让你能快速看到结果，我们先截取前15000个问题作为本次实践的语料库。
# 在你完全掌握流程后，可以回来删除或修改这行代码，以处理整个数据集。
corpus_subset = corpus[:15000]

print(f"为加速实践，我们使用一个包含 {len(corpus_subset)} 个问题的子集。")
print("\n语料库子集的前5个问题示例:")
# 打印看看我们的语料库是什么样的
for question in corpus_subset[:5]:
    print(question)

FileNotFoundError: [Errno 2] No such file or directory: 'questions.csv'

重新开始代码运行

In [6]:
# ===================================================================
# 步骤 1: 重新配置Kaggle API并下载数据
# ===================================================================
from google.colab import files
import os

# 检查 kaggle.json 是否已上传和配置，如果没有，则重新上传
if not os.path.exists("~/.kaggle/kaggle.json"):
    print('请上传您从Kaggle下载的 kaggle.json 文件：')
    uploaded = files.upload()

    if 'kaggle.json' in uploaded:
        print("kaggle.json 上传成功！正在配置环境...")
        !mkdir -p ~/.kaggle
        !mv kaggle.json ~/.kaggle/
        !chmod 600 ~/.kaggle/kaggle.json
    else:
        # 如果用户没有上传文件，就停止执行
        raise FileNotFoundError("未上传 kaggle.json，无法继续。请重新运行此单元格并上传文件。")

# 检查数据集是否已存在，如果不存在，则重新下载
if not os.path.exists("questions.csv"):
    print("未找到数据集 'questions.csv'，正在从Kaggle下载...")
    !kaggle datasets download -d quora/question-pairs-dataset
    !unzip -o question-pairs-dataset.zip
else:
    print("'questions.csv' 已存在，跳过下载。")


# ===================================================================
# 步骤 2: 加载数据、清洗并构建语料库
# ===================================================================
import pandas as pd

try:
    print("\n正在加载和处理数据...")
    df = pd.read_csv('questions.csv')

    # --- 清洗数据：处理缺失值 ---
    print(f"清洗前的数据总行数: {len(df)}")
    df.dropna(subset=['question1', 'question2'], inplace=True)
    print(f"清洗后的数据总行数: {len(df)}")

    # --- 构建语料库 (Corpus) ---
    all_questions = pd.concat([df['question1'], df['question2']])
    unique_questions = all_questions.unique()
    corpus = unique_questions.tolist()
    print(f"成功构建完整语料库！包含 {len(corpus)} 个不重复的问题。")

    # --- 截取部分语料库 ---
    corpus_subset = corpus[:15000]
    print(f"为加速实践，我们使用一个包含 {len(corpus_subset)} 个问题的子集。")
    print("\n语料库子集的前5个问题示例:")
    for question in corpus_subset[:5]:
        print(question)

    print("\n数据准备完成！可以进入下一步。")

except FileNotFoundError:
    print("\n错误：仍然找不到 'questions.csv'。请检查Kaggle下载步骤是否出错。")

请上传您从Kaggle下载的 kaggle.json 文件：


Saving kaggle.json to kaggle.json
kaggle.json 上传成功！正在配置环境...
未找到数据集 'questions.csv'，正在从Kaggle下载...
Dataset URL: https://www.kaggle.com/datasets/quora/question-pairs-dataset
License(s): other
  0% 0.00/20.8M [00:00<?, ?B/s]
100% 20.8M/20.8M [00:00<00:00, 1.16GB/s]
Archive:  question-pairs-dataset.zip
  inflating: questions.csv           

正在加载和处理数据...
清洗前的数据总行数: 404351
清洗后的数据总行数: 404348
成功构建完整语料库！包含 537386 个不重复的问题。
为加速实践，我们使用一个包含 15000 个问题的子集。

语料库子集的前5个问题示例:
What is the step by step guide to invest in share market in india?
What is the story of Kohinoor (Koh-i-Noor) Diamond?
How can I increase the speed of my internet connection while using a VPN?
Why am I mentally very lonely? How can I solve it?
Which one dissolve in water quikly sugar, salt, methane and carbon di oxide?

数据准备完成！可以进入下一步。


In [7]:
from sentence_transformers import SentenceTransformer
import numpy as np

# 1. 确保我们的AI模型已经加载
# 即使之前的单元格运行过，为了代码的独立性和可重用性，我们再次声明。
# 这能确保在任何情况下，'model' 变量都是可用的。
print("正在加载 SentenceTransformer 模型 'all-MiniLM-L6-v2'...")
model = SentenceTransformer('all-MiniLM-L6-v2')
print("模型加载完毕！")


# 2. 对语料库子集进行编码
# 这是计算密集型步骤，在Colab的GPU上大约需要1-2分钟。
print("\n开始对语料库进行编码... 请稍候。")

# model.encode() 是这个库的核心功能
# show_progress_bar=True 会显示一个进度条，让我们能直观地看到处理进度
corpus_embeddings = model.encode(corpus_subset, show_progress_bar=True)

print("编码完成！")


# 3. 检查编码结果的“形状”
# 这是一个非常重要的验证步骤
print(f"\n我们得到的嵌入向量矩阵的形状: {corpus_embeddings.shape}")

# 我们可以打印第一个问题和它对应的向量（只看前10个维度），来直观感受一下
print("\n第一个问题原文:")
print(corpus_subset[0])
print("\n它对应的嵌入向量（前10个维度）:")
print(corpus_embeddings[0][:10])

正在加载 SentenceTransformer 模型 'all-MiniLM-L6-v2'...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

模型加载完毕！

开始对语料库进行编码... 请稍候。


Batches:   0%|          | 0/469 [00:00<?, ?it/s]

编码完成！

我们得到的嵌入向量矩阵的形状: (15000, 384)

第一个问题原文:
What is the step by step guide to invest in share market in india?

它对应的嵌入向量（前10个维度）:
[ 0.0681499  -0.03966416 -0.06096719  0.00746603 -0.05872769  0.05477433
  0.01803629  0.02473968 -0.10618582 -0.02212325]


In [8]:
from sentence_transformers import util
import time

# 1. 定义一个你想搜索的查询（Query）
# 我们可以故意使用一些与语料库中不完全相同的词语，来测试模型的语义理解能力
query = "How do I learn programming in Python?"

print(f"用户查询: \"{query}\"")


# 2. 将查询也编码成一个向量
# 这一步至关重要：查询和语料库必须使用同一个模型进行编码，才能在同一个向量空间中进行比较
print("\n正在将查询编码为向量...")
query_embedding = model.encode(query)


# 3. 使用工具函数 util.semantic_search 执行搜索
print("正在语料库中进行语义搜索...")

# 记录搜索开始的时间
start_time = time.time()

# util.semantic_search 会高效地计算查询向量与所有语料库向量的相似度
# top_k=5 表示我们希望找到最相似的前5个结果
hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=5)

# 记录搜索结束的时间
end_time = time.time()

print(f"搜索完成！耗时: {end_time - start_time:.4f} 秒")

# hits 是一个列表的列表，因为可以同时搜索多个查询。我们只搜索了一个，所以取第一个元素。
hits = hits[0]


# 4. 优雅地打印搜索结果
print(f"\n与查询最相关的 Top 5 问题是:")
for hit in hits:
    # hit 是一个字典，包含 'corpus_id' 和 'score'
    # corpus_id 是匹配到的问题在原始列表 corpus_subset 中的索引
    # score 是该问题与查询的余弦相似度分数
    matched_question = corpus_subset[hit['corpus_id']]
    similarity_score = hit['score']

    print(f"- \"{matched_question}\" (相似度得分: {similarity_score:.4f})")

用户查询: "How do I learn programming in Python?"

正在将查询编码为向量...
正在语料库中进行语义搜索...
搜索完成！耗时: 0.0415 秒

与查询最相关的 Top 5 问题是:
- "How should I begin learning Python?" (相似度得分: 0.8696)
- "How should I start learning Python?" (相似度得分: 0.8581)
- "How do I learn Python systematically?" (相似度得分: 0.8390)
- "What's the best way to learn python on my own?" (相似度得分: 0.8167)
- "How easy is it to learn Python?" (相似度得分: 0.7779)
